In [68]:
'''
    For creating BST from a sorted list
'''

class Node:
    def __init__(self, x):
        self.val = x
        self.leftChild = None
        self.rightChild = None
        
def isBalanced(root):
    # Checks whether a BST is balanced or not

    if root == None:
        return True
    height_diff = getHeight(root.leftChild) - getHeight(root.rightChild)

    if abs(height_diff) > 1:
        return False
    else:
        return is_balanced(root.leftChild) and is_balanced(root.rightChild)
      
def getHeight(node):
    # Finds the height of any node in a BST
        if node == None:
            return 0
        return max(getHeight(node.leftChild), getHeight(node.rightChild)) + 1

def sortedArrayToBalancedBST(items):
    # Recursively calls itself to change a sorted list array into a Balanced binary tree.
    if items==[]:
        return None
    num_items = len(items)
    node = Node(items[num_items/2])
    
    node.leftChild = sortedArrayToBalancedBST(items[:num_items/2])
    node.rightChild = sortedArrayToBalancedBST(items[(num_items/2)+1:])

    return node

#Test
items = range(1, 32)
result = sortedArrayToBalancedBST(items)
print result.val
print isBalanced(result)
print getHeight(result)

16
True
5


In [69]:
import math

def node_height(bfs_index):
    '''
        Input:
            i : the index of the item in a BST placed in a breadth first order.
        Output:
            An integer that tells the height of the item in a BST
        Process:
            Number of elements at each height double in each increase of height.
            So, in Bits, the number increases shifts to 1 bit to left on each increase of height
            Thus, we move the bits of index to the left, and check how many shift it takes to become zero.
            The number of shifts is the height of the element in the array.      
    '''
    height = 0
    while bfs_index:
        bfs_index >>= 1 #Shifting the bfs_index to 1-bit left
        height += 1 
    return height
def hyperceil(i):
    '''
        Input: 
            i : the height of a BST or its subtree
        Output: 
            Returns the maximum height for the spliting for the lower part
        Process:
            Calls 'node_height' function to find the height of the given index and uses bit-wise operator
            to shift one left, to get the maximum height.
            
    '''
    return 1 << node_height(i - 1)



def bft_to_veb(bft_index, height):
    '''
        Process: 
            Takes in the bfs_index of an element and the height of BST and gives out its index in vEB layout
        Input:
            bft_index : The index of an element in Breadth First Search Order
            height : The height of the BST
        Output:
            An integer that says the index or position of the element in vEB layout 
    '''
        
    if height <= 2: #If height is less than too, than it the bfs_index will be same as its index
        return bft_index

    depth = int(math.ceil(math.log(bft_index, 2)))
        # Calculating the depth of the tree from index in Breadth First Search Order
        # Each depth would include double of the elements. For instance, 3 element in BFS would be in height 2,
        # So, finding the ceiling log on base 2 to find the height, 
    
    # Splitting the tree into two halves and finding height of upper tree and lower tree.
    bottom_height = hyperceil((height + 1) / 2)
    
    top_height = height - bottom_height

    if depth < top_height:
        # If depth of bfs_index is lower than the height of top_tree recursviely call bfs_to_veb
        return bft_to_veb(bft_index, top_height)

    # If the element is in the lower sub_tree or lower half of the split, then find the sub_tree depth
    
    subtree_depth = depth - top_height # Depth of the lower half of the tree split.
    
    subtree_root = bft_index >> subtree_depth
    # Since we are looking at lower split, we move the root to lower part by moving the root to lower depths.
    # The lower depth is achieved by moving the index to left by the depth 
 
    num_subtrees = 1 << top_height #Number of sub-trees of each node
    # Finding the bfs_index relative to the new_subtree_root by deleting common bits and addint 1.
    bft_index &= (1 << subtree_depth) - 1 
    bft_index |= (1 << subtree_depth) 

    # Calculating all the nodes before this node and position wihtin the subtree.
    
    # Elements in both split would contain one more element. 
    # Shifting bottom_height by 1 bit left, to increase by 2 and subtracting 1.
    subtree_size = (1 << bottom_height) - 1
    toptree_size = (1 << top_height) - 1 

    # Counting all the nodes before the current node.
    # Consists of element in the top_tree and the siblings before the sub_tree root
    
    prior_length = toptree_size + (subtree_root & (num_subtrees - 1)) * subtree_size
    return prior_length + bft_to_veb(bft_index, bottom_height)

# Test
print bft_to_veb(5, 5)

4


In [82]:
def breadthFirstTraversal(root):
    '''
        Process: 
            Performs Breadth First Travesal on BST and stores the elements in an array in BFT order.
        Input:
            root: The root node of BST
        Output:
            Returns two arrays, with values and nodes of the BST in Breadth First Order Traversal
    '''
    if root is None:
        return
     
    bft_val_array = []
    bft_node_array = []
   
    # Create an empty queue for breadth first traversal
    queue = []
 
    queue.append(root)
 
    while(len(queue) > 0):
        # Add to the queue to the arrays and remove it from queue
        bft_val_array.append(queue[0].val)
        bft_node_array.append(queue[0])
        node = queue.pop(0)
 
        #Add left child to the queue
        if node.leftChild is not None:
            queue.append(node.leftChild)
 
        # Add right child to the queue
        if node.rightChild is not None:
            queue.append(node.rightChild)
    return bft_val_array, bft_node_array



def bft_to_veb_layout_converter(height, bft_val_array, bft_node_array):
    '''
        Process: 
            For each element of BST, it calls bft_to_veb function to find its index in vEB layout,
            by passing the index of the element in Breadth First Traversal Order.
        Input:
            bft_val_array : An array with all the values of nodes of BST in breadth first traversal order 
            bft_node_array : An array with all the nodes of BST in breadth first traversal order
            height : Height of the BST
        Output:
            Returns two arrays, with values and nodes of the BST in vEB layout
    '''
    veb_val_array = [0 for _ in range(len(bft_val_array))]
        # Array to store all the values of nodes of BST in vEB layout order
    veb_node_array = [0 for _ in range(len(bft_node_array))]
        # Array to store all the values of nodes of BST in vEB layout order
        
    for i in range(len(bft_val_array)):
        veb_index = bft_to_veb(i + 1, height)
        veb_val_array[veb_index - 1] = bft_val_array[i]
        veb_node_array[veb_index - 1] = bft_node_array[i]
    
    return veb_val_array, veb_node_array

In [83]:
def main():
    items = range(1, 32)
    result = sortedArrayToBalancedBST(items)
    print result.val
    print isBalanced(result)
    print getHeight(result)
    bft_val_array, bft_node_array = breadthFirstTraversal(result) 
    veb_val_array, veb_node_array =  bft_to_veb_layout_converter(getHeight(result), bft_val_array, bft_node_array)
    
    print "In Breadth First Order"
    print bft_val_array
    
    print "\nIn vEB layout"
    for node in veb_node_array:
        print node.val, 

main()

16
True
5
In Breadth First Order
[16, 8, 24, 4, 12, 20, 28, 2, 6, 10, 14, 18, 22, 26, 30, 1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31]

In vEB layout
16 8 4 12 2 1 3 6 5 7 10 9 11 14 13 15 24 20 28 18 17 19 22 21 23 26 25 27 30 29 31
